# Setup

In [1]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

auth.authenticate_user()
print('Authenticated')

project_id = "dummy-400508" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


## Test Query

In [2]:
df = client.query('''
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
''').to_dataframe()

df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027)
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008)
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.00,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014)
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
...,...,...,...,...,...,...,...,...,...,...,...
9281,FI-2165,I-407,2021-04-06 16:00:00+00:00,2021-04-06 16:00:00+00:00,1.287472,5.25,1.31,Credit Card,Federway Int.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259)
9282,FI-3890,I-580,2022-11-13 15:30:00+00:00,2022-11-13 15:45:00+00:00,2.574944,7.75,1.31,Credit Card,Federway Int.,POINT (-87.6173580061 41.859349715),POINT (-87.6209929134 41.8849871918)
9283,AI-11438,AI-527,2021-05-14 15:30:00+00:00,2021-05-14 15:30:00+00:00,1.448406,5.25,1.31,Credit Card,Atomic Inc.,POINT (-87.6209929134 41.8849871918),POINT (-87.6188683546 41.8909220259)
9284,AI-11220,A-842,2021-04-30 18:00:00+00:00,2021-04-30 18:00:00+00:00,1.770274,6.25,1.56,Credit Card,Atomic Inc.,POINT (-87.6327464887 41.8809944707),POINT (-87.657005027 41.8790669938)


# SQL

## 1. Apa saja nama perusahaan yang terdapat dalam data?

In [3]:
df = client.query('''
SELECT distinct(company)
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
''').to_dataframe()

df

,company
0,Deluc Porter
1,Synctaxi Inc.
2,Atomic Inc.
3,Federway Int.
4,Rocket Cab Auto


Terdapat 5 nama `company` yang terdaftar: `Deluc Porter`, `Synctaxi Inc.`, `Atomic Inc.`, `Federway Int.`, dan `Rocket Cab Auto`

## 2. Berapa nilai minimum dan maksimum dari waktu pengambilan data?

In [4]:
df = client.query('''
SELECT min(trip_start_timestamp) as min, max(trip_start_timestamp) as max
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
''').to_dataframe()

df

,min,max
0,2021-01-02 11:00:00+00:00,2022-12-31 14:15:00+00:00


- trip_start minimum adalah: `2021-01-02 11:00:00+00:00`
- trip_start maximum adalah: `2022-12-31 14:15:00+00:00`

## 3. Bandingkan jumlah banyaknya perjalanan antara seluruh perusahaan pada tahun 2022?

In [5]:
df = client.query('''
SELECT company, COUNT(1) as trip_count
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
WHERE EXTRACT(year from trip_start_timestamp) = 2022
GROUP BY company
''').to_dataframe()

df

,company,trip_count
0,Deluc Porter,1050
1,Synctaxi Inc.,1341
2,Federway Int.,1059
3,Rocket Cab Auto,77


Berikut adalah jumlah banyaknya trip dari setiap `company`
- Deluc Porter: 1050
- Synctaxi Inc.: 1341
- Atomic Inc.: 0
- Federway Int.: 1059
- Rocket Cab Auto: 77

## 4. Perusahaan mana yang memiliki tarif perjalanan per km termurah?

In [11]:
df = client.query('''
-- membuat pivot table dari hasil subquery di FROM, yang menampilkan nama company dan min. tarif/km
SELECT company, fare_per_km
FROM (
  -- mendapatkan nama company dan rata-rata tarif/km dari setiap company
  SELECT company, MIN(fare/trip_km) as fare_per_km
  FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
  WHERE fare > 0 -- exclude fare 0
  GROUP BY company
)
-- mencari rata-rata tarif/km paling rendah dengan menggunakan hasil subquery
WHERE fare_per_km = (
  SELECT MIN(fare_per_km)
  FROM (
    SELECT company, MIN(fare/trip_km) as fare_per_km
    FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
    WHERE fare > 0 -- exclude fare 0
    GROUP BY company
  )
)
''').to_dataframe()

df

,company,fare_per_km
0,Federway Int.,0.000875


Perusahaan yang memiliki rata-rata tarif per km terendah adalah: `Federway Int.`

# Pandas

In [7]:
df = client.query('''
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
''').to_dataframe()

df.head(1)

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.0,0.0,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)


## 1. buat kolom baru, dengan nama Keterangan_Waktu.
Kolom ini berisikan label yang menunjukkan representasi keterangan waktu berdasarkan jam dengan ketentuan berikut:

|Keterangan Waktu|Jam
|---|---
|Pagi|04:00 - 09:59
|Siang|10:00 - 15:59
|Sore|16:00 - 18:59
|Malam|19:00 - 23:59
|Dini Hari|00:00 - 03:59

In [8]:
df['Hour'] = df['trip_start_timestamp'].dt.hour

# temp storage untuk series Keterangan_Waktu
keterangan = []

for time in df['Hour']:
  if time < 4:
    keterangan.append("Dini hari")
  elif time < 10:
    keterangan.append("Pagi")
  elif time < 16:
    keterangan.append("Siang")
  elif time < 19:
    keterangan.append("Sore")
  else:
    keterangan.append("Malam")


df["Keterangan_Waktu"] = keterangan

df.head()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location,Hour,Keterangan_Waktu
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.057746,9.00,0.0,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),19,Malam
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.082354,35.50,0.0,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027),15,Siang
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.242030,14.00,0.0,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008),19,Malam
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.369826,37.25,0.0,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014),13,Siang
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.218680,9.00,0.0,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383),15,Siang


## 2. Berapa total jarak perjalanan yang dicapai perusahaan Federway Int. pada tahun 2022? (Gunakan pandas query untuk menjawab soal no. 2)

In [9]:
# melakukan filter sesuai soal
df_filter = df[(df['company'] == 'Federway Int.') & (df['trip_start_timestamp'].dt.year == 2022)]

# temp storage untuk menyimpan hasil total jarak perjalanan
total_trip = 0

for trip in df_filter['trip_km']:
  # increment total_trip dengan setiap angka trip_km pada series 'trip_km' dari df_filter
  total_trip += trip

print("total trip dari Federway Int. pada tahun 2022 adalah:", total_trip)

total trip dari Federway Int. pada tahun 2022 adalah: 4359.241349999974
